# Laboratorio 1 – Comparación de Bases de Datos NoSQL (MongoDB, Redis, HBase)

**Objetivo:**
Este laboratorio tiene como objetivo montar servidores de **MongoDB**, **Redis** y **HBase** en contenedores Docker, y luego interactuar con ellos desde Python para:

1. Cargar un dataset real de compras electrónicas.
2. Ejecutar consultas específicas en cada base de datos:
   - ¿Cuál es la **categoría** más vendida?
   - ¿Cuál **marca** generó más ingresos brutos?
   - ¿Qué **mes** tuvo más ventas? (en UTC)
3. Medir y comparar los tiempos de ejecución entre las tres bases de datos.

**Dataset utilizado:**
[Ecommerce Purchase History from Electronics Store](https://www.kaggle.com/datasets/mkechinov/ecommerce-purchase-history-from-electronics-store)

**Pasos principales:**
1. Preparar el entorno de Python en esta máquina virtual.
2. Descargar y preparar el dataset.
3. Insertar datos en **MongoDB**, **Redis** y **HBase**.
4. Ejecutar las consultas en cada base de datos.
5. Analizar los tiempos de respuesta.

---


In [1]:
# Instalación de librerías necesarias
!pip3 install --user pandas pymongo redis happybase thriftpy2 kaggle


## 📥 Descarga del Dataset desde Kaggle

Para trabajar con el dataset real, utilizaremos la API de Kaggle.  
Esto requiere que tengamos un **token de acceso** que puedes obtener en tu cuenta de Kaggle:

1. Inicia sesión en [Kaggle](https://www.kaggle.com/).
2. Ve a **Account** → **Create New API Token**.
3. Esto descargará un archivo llamado `kaggle.json`.
4. Lo subiremos a esta máquina en la carpeta `~/.kaggle/`.

Este archivo contiene:
- `username`: tu usuario en Kaggle.
- `key`: clave de acceso para autenticación.


In [2]:
import os

# Crear carpeta .kaggle si no existe
os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)

print("✅ Carpeta .kaggle lista. Sube tu archivo kaggle.json a esta ruta en la VM:")

# Mostrar ruta destino
print(os.path.expanduser("~/.kaggle/kaggle.json"))


✅ Carpeta .kaggle lista. Sube tu archivo kaggle.json a esta ruta en la VM:
/home/azureuser/.kaggle/kaggle.json


## 🔓 Ajustar permisos y descargar dataset

Después de subir el archivo `kaggle.json` a la carpeta `.kaggle/` debemos ajustar los permisos para que sea seguro.  
Luego utilizaremos la librería `kaggle` para descargar el dataset directamente en esta máquina.

El dataset se guardará en la carpeta `datasets/ecommerce/`.


In [3]:
# Ajustar permisos del token Kaggle
!chmod 600 ~/.kaggle/kaggle.json

# Descargar el dataset desde Kaggle
!kaggle datasets download -d mkechinov/ecommerce-purchase-history-from-electronics-store -p ./datasets/ecommerce --unzip

print("✅ Dataset descargado y descomprimido en ./datasets/ecommerce")


Dataset URL: https://www.kaggle.com/datasets/mkechinov/ecommerce-purchase-history-from-electronics-store
License(s): copyright-authors
 71%|███████████████████████████▊           | 36.0M/50.5M [00:00<00:00, 377MB/s]
100%|███████████████████████████████████████| 50.5M/50.5M [00:00<00:00, 377MB/s]
✅ Dataset descargado y descomprimido en ./datasets/ecommerce


In [4]:
import os

os.listdir("./datasets/ecommerce")


['kz.csv']

## 📂 Carga y vista previa del Dataset

El dataset descargado desde Kaggle contiene información sobre el historial de compras en una tienda de electrónica.  
El archivo descargado se llama `kz.csv` y lo cargaremos usando **Pandas** para inspeccionarlo antes de insertarlo en las bases de datos.

Pasos:
1. Leer el archivo CSV.
2. Mostrar las primeras filas para validar el contenido.
3. Revisar la estructura de las columnas.


In [5]:
import pandas as pd

# Ruta al archivo descargado
file_path = "./datasets/ecommerce/kz.csv"

# Cargar el dataset
df = pd.read_csv(file_path)

# Vista previa de las primeras filas
df.head()


,event_time,order_id,product_id,category_id,category_code,brand,price,user_id
0,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
1,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
2,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
3,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
4,2020-04-24 19:16:21 UTC,2294584263154074236,2273948316817424439,2.268105e+18,NaN,karcher,217.57,1.515916e+18


## 📊 Inspección de la estructura del Dataset

Antes de proceder a insertar los datos en las bases de datos, es importante entender su estructura y contenido.

Pasos:
1. Revisar la información general (`info()`) para conocer número de filas, columnas y tipos de datos.
2. Verificar estadísticas descriptivas (`describe()`) para entender rangos y distribución de datos numéricos.
3. Detectar posibles valores nulos o faltantes.


In [6]:
# Información general del dataset
print("=== Información General del Dataset ===")
print(df.info())

# Estadísticas descriptivas de columnas numéricas
print("\n=== Estadísticas Descriptivas ===")
print(df.describe())

# Conteo de valores nulos por columna
print("\n=== Valores Nulos por Columna ===")
print(df.isnull().sum())


=== Información General del Dataset ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2633521 entries, 0 to 2633520
Data columns (total 8 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   order_id       int64  
 2   product_id     int64  
 3   category_id    float64
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        float64
dtypes: float64(3), int64(2), object(3)
memory usage: 160.7+ MB
None

=== Estadísticas Descriptivas ===
           order_id    product_id   category_id         price       user_id
count  2.633521e+06  2.633521e+06  2.201567e+06  2.201567e+06  5.641690e+05
mean   2.361783e+18  1.674080e+18  2.273827e+18  1.540932e+02  1.515916e+18
std    1.716538e+16  3.102249e+17  2.353247e+16  2.419421e+02  2.377083e+07
min    2.294360e+18  1.515966e+18  2.268105e+18  0.000000e+00  1.515916e+18
25%    2.348807e+18  1.515966e+18  2.268105e+18  1.456000e+01  1.515916e+18
50%    2

## Limpieza y Preprocesamiento del Dataset

Antes de insertar en las bases de datos, limpiamos y normalizamos la información para:
- Manejar valores nulos.
- Convertir `event_time` a formato `datetime` UTC.
- Garantizar compatibilidad con los formatos de cada base de datos.


In [7]:
import pandas as pd
import numpy as np
import time

CSV_PATH = "./datasets/ecommerce/kz.csv"
CHUNK_SIZE = 100_000

start_time = time.time()

for i, chunk in enumerate(pd.read_csv(CSV_PATH, chunksize=CHUNK_SIZE)):
    print(f"🔄 Procesando chunk {i+1}...")

    # Conversión de event_time a datetime UTC
    chunk["event_time"] = pd.to_datetime(chunk["event_time"], errors="coerce", utc=True)

    # Manejo de valores nulos
    chunk["category_code"] = chunk["category_code"].fillna("")
    chunk["brand"] = chunk["brand"].fillna("")
    chunk["category_id"] = chunk["category_id"].fillna(0).astype("int64")
    chunk["price"] = chunk["price"].fillna(0).astype("float64")
    chunk["user_id"] = chunk["user_id"].fillna(0).astype("int64")

    # Aquí podríamos insertar directamente en MongoDB, Redis o HBase
    # insert_to_mongo(chunk)  # Ejemplo
    print(f"✅ Chunk {i+1} limpio y listo ({len(chunk)} filas)")

end_time = time.time()
print(f"⏱ Tiempo total de limpieza: {end_time - start_time:.2f} segundos")


🔄 Procesando chunk 1...
✅ Chunk 1 limpio y listo (100000 filas)
🔄 Procesando chunk 2...
✅ Chunk 2 limpio y listo (100000 filas)
🔄 Procesando chunk 3...
✅ Chunk 3 limpio y listo (100000 filas)
🔄 Procesando chunk 4...
✅ Chunk 4 limpio y listo (100000 filas)
🔄 Procesando chunk 5...
✅ Chunk 5 limpio y listo (100000 filas)
🔄 Procesando chunk 6...
✅ Chunk 6 limpio y listo (100000 filas)
🔄 Procesando chunk 7...
✅ Chunk 7 limpio y listo (100000 filas)
🔄 Procesando chunk 8...
✅ Chunk 8 limpio y listo (100000 filas)
🔄 Procesando chunk 9...
✅ Chunk 9 limpio y listo (100000 filas)
🔄 Procesando chunk 10...
✅ Chunk 10 limpio y listo (100000 filas)
🔄 Procesando chunk 11...
✅ Chunk 11 limpio y listo (100000 filas)
🔄 Procesando chunk 12...
✅ Chunk 12 limpio y listo (100000 filas)
🔄 Procesando chunk 13...
✅ Chunk 13 limpio y listo (100000 filas)
🔄 Procesando chunk 14...
✅ Chunk 14 limpio y listo (100000 filas)
🔄 Procesando chunk 15...
✅ Chunk 15 limpio y listo (100000 filas)
🔄 Procesando chunk 16...
✅ C

In [8]:
# Verificación de valores nulos en el dataset limpio
print("=== Valores Nulos por Columna (Dataset Limpio) ===")
null_counts = df.isnull().sum()
print(null_counts)


=== Valores Nulos por Columna (Dataset Limpio) ===
event_time             0
order_id               0
product_id             0
category_id       431954
category_code     612202
brand             506005
price             431954
user_id          2069352
dtype: int64


## 📥 Inserción de datos en MongoDB

Conexión y carga de datos en MongoDB usando `pymongo`.

Pasos:
1. Conectarse al servidor de MongoDB.
2. Seleccionar la base de datos (`ecommerce_db`) y la colección (`purchases`).
3. Convertir el DataFrame de Pandas a diccionarios (`to_dict('records')`).
4. Insertar los datos con `insert_many()`.

> ⚠️ **Nota:** El contenedor MongoDB debe estar corriendo y accesible en el puerto `27017`.


In [9]:
import time
import pandas as pd
from pymongo import MongoClient

# ==== Parámetros de conexión ====
MONGO_USER = "admin"
MONGO_PASS = "pass"
MONGO_HOST = "localhost"   # Si Jupyter y MongoDB están en la misma VM
MONGO_PORT = 27017
DB_NAME = "ecommerce_db"
COLLECTION_NAME = "purchases"

# ==== Conexión a MongoDB ====
client = MongoClient(f"mongodb://{MONGO_USER}:{MONGO_PASS}@{MONGO_HOST}:{MONGO_PORT}/?authSource=admin")
db = client[DB_NAME]
collection = db[COLLECTION_NAME]

# ==== Limpieza de la colección ====
collection.drop()
print("🧹 Colección limpiada antes de la inserción.\n")

# ==== Parámetros de carga ====
CHUNK_SIZE = 100_000  # Número de registros por bloque
total_rows = len(df)  # df debe estar previamente cargado con el dataset
total_start_time = time.time()

# ==== Inserción por bloques ====
for i in range(0, total_rows, CHUNK_SIZE):
    chunk_start_time = time.time()

    # Seleccionar un bloque de datos
    df_chunk = df.iloc[i:i + CHUNK_SIZE]
    data_dict = df_chunk.to_dict("records")

    # Insertar en MongoDB
    result = collection.insert_many(data_dict)

    chunk_end_time = time.time()
    print(f"✅ Bloque {i // CHUNK_SIZE + 1}: {len(result.inserted_ids)} registros "
          f"({chunk_end_time - chunk_start_time:.2f} seg)")

# ==== Tiempo total ====
total_end_time = time.time()
print(f"\n⏱ Tiempo total: {total_end_time - total_start_time:.2f} seg")
print(f"📊 Total documentos insertados: {collection.count_documents({})}")


🧹 Colección limpiada antes de la inserción.

✅ Bloque 1: 100000 registros (4.23 seg)
✅ Bloque 2: 100000 registros (4.72 seg)
✅ Bloque 3: 100000 registros (4.35 seg)
✅ Bloque 4: 100000 registros (4.28 seg)
✅ Bloque 5: 100000 registros (4.59 seg)
✅ Bloque 6: 100000 registros (4.16 seg)
✅ Bloque 7: 100000 registros (4.12 seg)
✅ Bloque 8: 100000 registros (4.49 seg)
✅ Bloque 9: 100000 registros (4.16 seg)
✅ Bloque 10: 100000 registros (4.07 seg)
✅ Bloque 11: 100000 registros (4.50 seg)
✅ Bloque 12: 100000 registros (4.19 seg)
✅ Bloque 13: 100000 registros (4.13 seg)
✅ Bloque 14: 100000 registros (4.59 seg)
✅ Bloque 15: 100000 registros (4.14 seg)
✅ Bloque 16: 100000 registros (4.37 seg)
✅ Bloque 17: 100000 registros (4.21 seg)
✅ Bloque 18: 100000 registros (4.31 seg)
✅ Bloque 19: 100000 registros (4.29 seg)
✅ Bloque 20: 100000 registros (4.25 seg)
✅ Bloque 21: 100000 registros (4.25 seg)
✅ Bloque 22: 100000 registros (4.62 seg)
✅ Bloque 23: 100000 registros (4.24 seg)
✅ Bloque 24: 100000 r

## Carga del dataset en Redis

En esta sección conectaremos con Redis desde Python utilizando la librería `redis-py`.
Cargaremos los datos del dataset en formato **Hash** (`HSET`), de forma que cada
registro tendrá una clave única y todos sus campos estarán agrupados en una misma estructura.

### Estrategia:
- Usaremos la conexión al contenedor Redis que corre en la misma VM (`localhost`).
- Guardaremos los datos como:
  - **Clave:** `purchase:<id>`
  - **Campos:** `category`, `brand`, `price`, `purchase_date`, etc.
- Cargaremos en bloques de **100 000 registros** para no saturar la memoria.
- Antes de insertar, limpiaremos el espacio de claves relacionadas con `purchase:` para evitar duplicados.
- Mediremos el tiempo por bloque y el tiempo total.


In [10]:
import redis
import pandas as pd
import time

# ==== Parámetros de conexión a Redis ====
redis_host = "localhost"
redis_port = 6379
redis_pass = None  # Cambiar si se configuró contraseña

# ==== Conexión ====
r = redis.Redis(host=redis_host, port=redis_port, password=redis_pass, decode_responses=True)

# ==== Verificación de conexión ====
try:
    r.ping()
    print("✅ Conectado a Redis")
except redis.exceptions.ConnectionError as e:
    raise SystemExit(f"❌ No se pudo conectar a Redis: {e}")

# ==== Ruta del dataset ====
dataset_path = "./datasets/ecommerce/kz.csv"

# ==== Cargar en DataFrame ====
df = pd.read_csv(dataset_path)

# ==== Parámetros de carga ====
CHUNK_SIZE = 100_000
total_rows = len(df)
print(f"📦 Total de registros en dataset: {total_rows:,}")

# ==== Limpieza previa de datos ====
keys_to_delete = r.keys("purchase:*")
if keys_to_delete:
    for i in range(0, len(keys_to_delete), 10_000):  # evitar saturar delete
        r.delete(*keys_to_delete[i:i+10_000])
    print(f"🧹 Eliminadas {len(keys_to_delete):,} claves antiguas en Redis")

# ==== Inserción por bloques ====
start_total = time.time()
for i in range(0, total_rows, CHUNK_SIZE):
    chunk = df.iloc[i:i + CHUNK_SIZE]
    start_chunk = time.time()

    pipe = r.pipeline(transaction=False)
    for idx, row in chunk.iterrows():
        key = f"purchase:{idx}"
        pipe.hset(key, mapping={k: ("" if pd.isna(v) else str(v)) for k, v in row.to_dict().items()})

    pipe.execute()
    elapsed_chunk = time.time() - start_chunk
    print(f"✅ Bloque {i//CHUNK_SIZE + 1}: {len(chunk):,} registros en {elapsed_chunk:.2f} seg")

elapsed_total = time.time() - start_total
print(f"🏁 Inserción total completada en {elapsed_total:.2f} segundos")


✅ Conectado a Redis
📦 Total de registros en dataset: 2,633,521
🧹 Eliminadas 2,633,521 claves antiguas en Redis
✅ Bloque 1: 100,000 registros en 42.05 seg
✅ Bloque 2: 100,000 registros en 40.58 seg
✅ Bloque 3: 100,000 registros en 40.79 seg
✅ Bloque 4: 100,000 registros en 40.61 seg
✅ Bloque 5: 100,000 registros en 40.09 seg
✅ Bloque 6: 100,000 registros en 41.15 seg
✅ Bloque 7: 100,000 registros en 41.36 seg
✅ Bloque 8: 100,000 registros en 41.28 seg
✅ Bloque 9: 100,000 registros en 41.36 seg
✅ Bloque 10: 100,000 registros en 41.18 seg
✅ Bloque 11: 100,000 registros en 41.31 seg
✅ Bloque 12: 100,000 registros en 41.40 seg
✅ Bloque 13: 100,000 registros en 41.06 seg
✅ Bloque 14: 100,000 registros en 41.12 seg
✅ Bloque 15: 100,000 registros en 40.68 seg
✅ Bloque 16: 100,000 registros en 41.31 seg
✅ Bloque 17: 100,000 registros en 40.84 seg
✅ Bloque 18: 100,000 registros en 41.65 seg
✅ Bloque 19: 100,000 registros en 41.26 seg
✅ Bloque 20: 100,000 registros en 41.48 seg
✅ Bloque 21: 100,0

## 🗄️ Carga del Dataset en HBase

En esta sección insertaremos el dataset limpio en **HBase**.

---

### ⚙️ Configuración de la prueba
- **Dataset:** `kz.csv` (2 633 521 registros)
- **Bloques de inserción:** 100 000 registros (último bloque menor)
- **HBase:** Contenedor Docker (`hbase:latest`) ejecutando en la misma VM
- **Conexión:** Usaremos la librería `happybase` vía `thriftpy2` para comunicación con el servidor Thrift de HBase.
- **VM:** `Standard_A4m_v2` — 8 vCPU, 32 GB RAM

---

### 📝 Consideraciones
1. Antes de cargar datos, nos aseguraremos de que **la tabla no exista** o **se vacíe** si ya existe, para evitar duplicados.
2. Cada registro se insertará como **una fila** en HBase, usando `order_id` como **row key**.
3. Usaremos `batch()` para mejorar el rendimiento y reducir el overhead de conexión.

---


In [11]:
import pandas as pd
import happybase
import time

# ==== Parámetros de conexión ====
HBASE_HOST = "localhost"
HBASE_PORT = 9090   # Puerto del servicio Thrift de HBase
TABLE_NAME = "purchases"

# ==== Conectar a HBase ====
connection = happybase.Connection(host=HBASE_HOST, port=HBASE_PORT)
connection.open()

print("✅ Conectado a HBase")

# ==== Crear tabla si no existe ====
families = {'cf': dict()}  # 'cf' = column family
if TABLE_NAME.encode() not in connection.tables():
    connection.create_table(TABLE_NAME, families)
    print(f"🆕 Tabla creada: {TABLE_NAME}")
else:
    print(f"📄 La tabla '{TABLE_NAME}' ya existe.")

table = connection.table(TABLE_NAME)

# ==== Limpieza previa ====
print("🧹 Limpiando registros antiguos de la tabla...")
# Nota: happybase no tiene truncate directo, así que borramos fila a fila
for key, _ in table.scan():
    table.delete(key)
print("🧹 Tabla vacía.\n")

# ==== Cargar dataset ====
dataset_path = "./datasets/ecommerce/kz.csv"
df = pd.read_csv(dataset_path)

CHUNK_SIZE = 100_000
total_rows = len(df)
print(f"📦 Total de registros en dataset: {total_rows:,}")

total_start_time = time.time()

for i in range(0, total_rows, CHUNK_SIZE):
    chunk = df.iloc[i:i + CHUNK_SIZE]
    start_chunk = time.time()

    # Usar batch para inserción rápida
    with table.batch(batch_size=CHUNK_SIZE) as b:
        for _, row in chunk.iterrows():
            row_key = str(row["order_id"]).encode()
            b.put(row_key, {
                b"cf:event_time": str(row["event_time"]).encode(),
                b"cf:product_id": str(row["product_id"]).encode(),
                b"cf:category_id": str(row["category_id"]).encode(),
                b"cf:category_code": str(row["category_code"]).encode() if pd.notna(row["category_code"]) else b"",
                b"cf:brand": str(row["brand"]).encode() if pd.notna(row["brand"]) else b"",
                b"cf:price": str(row["price"]).encode(),
                b"cf:user_id": str(row["user_id"]).encode() if pd.notna(row["user_id"]) else b""
            })

    elapsed_chunk = time.time() - start_chunk
    print(f"✅ Bloque {i//CHUNK_SIZE + 1}: {len(chunk):,} registros en {elapsed_chunk:.2f} segundos")

total_elapsed = time.time() - total_start_time
print(f"\n🏁 Inserción total completada en {total_elapsed:.2f} segundos")


✅ Conectado a HBase
📄 La tabla 'purchases' ya existe.
🧹 Limpiando registros antiguos de la tabla...
🧹 Tabla vacía.

📦 Total de registros en dataset: 2,633,521
✅ Bloque 1: 100,000 registros en 49.28 segundos
✅ Bloque 2: 100,000 registros en 47.64 segundos
✅ Bloque 3: 100,000 registros en 47.18 segundos
✅ Bloque 4: 100,000 registros en 47.28 segundos
✅ Bloque 5: 100,000 registros en 47.85 segundos
✅ Bloque 6: 100,000 registros en 47.16 segundos
✅ Bloque 7: 100,000 registros en 47.60 segundos
✅ Bloque 8: 100,000 registros en 47.45 segundos
✅ Bloque 9: 100,000 registros en 48.55 segundos
✅ Bloque 10: 100,000 registros en 49.36 segundos
✅ Bloque 11: 100,000 registros en 48.99 segundos
✅ Bloque 12: 100,000 registros en 47.98 segundos
✅ Bloque 13: 100,000 registros en 48.40 segundos
✅ Bloque 14: 100,000 registros en 46.95 segundos
✅ Bloque 15: 100,000 registros en 46.82 segundos
✅ Bloque 16: 100,000 registros en 47.52 segundos
✅ Bloque 17: 100,000 registros en 47.99 segundos
✅ Bloque 18: 100,

## 1️⃣ Consulta: Categoría más vendida

**Objetivo:** Determinar cuál categoría (`category_code`) aparece con mayor frecuencia en las transacciones.

Se ejecutará la misma consulta en:
- **MongoDB**
- **Redis**
- **HBase**

Además, se medirá el tiempo de ejecución en cada base de datos para su posterior comparación.


In [12]:
import time
from pymongo import MongoClient

# Conexión MongoDB
mongo_client = MongoClient(f"mongodb://{MONGO_USER}:{MONGO_PASS}@{MONGO_HOST}:{MONGO_PORT}/?authSource=admin")
mongo_db = mongo_client[DB_NAME]
mongo_col = mongo_db[COLLECTION_NAME]

start_time = time.time()
pipeline = [
    {"$group": {"_id": "$category_code", "total": {"$sum": 1}}},
    {"$sort": {"total": -1}},
    {"$limit": 1}
]
result = list(mongo_col.aggregate(pipeline))
elapsed_time = time.time() - start_time

print(f"📊 Categoría más vendida (MongoDB): {result[0]['_id']} con {result[0]['total']:,} ventas")
print(f"⏱ Tiempo: {elapsed_time:.4f} segundos")


📊 Categoría más vendida (MongoDB): nan con 612,202 ventas
⏱ Tiempo: 8.5758 segundos


In [13]:
import redis
import time
from collections import Counter

# Conexión Redis
r = redis.Redis(host=redis_host, port=redis_port, password=redis_pass)

start_time = time.time()
category_counter = Counter()

for key in r.scan_iter("purchase:*"):
    category_code = r.hget(key, "category_code")
    if category_code:
        category_counter[category_code.decode()] += 1

top_category, top_count = category_counter.most_common(1)[0]
elapsed_time = time.time() - start_time

print(f"📊 Categoría más vendida (Redis): {top_category} con {top_count:,} ventas")
print(f"⏱ Tiempo: {elapsed_time:.4f} segundos")


📊 Categoría más vendida (Redis): electronics.smartphone con 357,682 ventas
⏱ Tiempo: 1238.2801 segundos


In [35]:
import happybase
import time
from collections import Counter

# Conexión HBase
connection = happybase.Connection(host="localhost", port=9090)
table = connection.table("purchases")

start_time = time.time()
category_counter = Counter()

for key, data in table.scan():
    category_code = data.get(b"cf:category_code")
    if category_code:
        category_counter[category_code.decode()] += 1

top_category, top_count = category_counter.most_common(1)[0]
elapsed_time = time.time() - start_time

print(f"📊 Categoría más vendida (HBase): {top_category} con {top_count:,} ventas")
print(f"⏱ Tiempo: {elapsed_time:.4f} segundos")


📊 Categoría más vendida (HBase): electronics.smartphone con 213,002 ventas
⏱ Tiempo: 173.2701 segundos


## 2️⃣ Consulta: Marca que generó más ingresos brutos

**Objetivo:** Determinar cuál `brand` generó la mayor suma de `price`.

Se ejecutará en:
- **MongoDB**
- **Redis**
- **HBase**


In [15]:
start_time = time.time()
pipeline = [
    {"$group": {"_id": "$brand", "total_ingresos": {"$sum": "$price"}}},
    {"$sort": {"total_ingresos": -1}},
    {"$limit": 1}
]
result = list(mongo_col.aggregate(pipeline))
elapsed_time = time.time() - start_time

print(f"💰 Marca con más ingresos (MongoDB): {result[0]['_id']} con ${result[0]['total_ingresos']:.2f}")
print(f"⏱ Tiempo: {elapsed_time:.4f} segundos")


💰 Marca con más ingresos (MongoDB): samsung con $90052821.66
⏱ Tiempo: 9.7873 segundos


In [16]:
start_time = time.time()
brand_revenue = Counter()

for key in r.scan_iter("purchase:*"):
    brand = r.hget(key, "brand")
    price = r.hget(key, "price")
    if brand and price:
        try:
            brand_revenue[brand.decode()] += float(price)
        except:
            pass

top_brand, top_revenue = brand_revenue.most_common(1)[0]
elapsed_time = time.time() - start_time

print(f"💰 Marca con más ingresos (Redis): {top_brand} con ${top_revenue:,.2f}")
print(f"⏱ Tiempo: {elapsed_time:.4f} segundos")


💰 Marca con más ingresos (Redis): samsung con $90,052,821.66
⏱ Tiempo: 2306.0426 segundos


In [36]:
from collections import Counter
import time

brand_revenue = Counter()
start_time = time.time()

# Configurar tamaño de bloque para scan
BATCH_SIZE = 5000

# Control de filas procesadas
rows_processed = 0
last_key = None

while True:
    # Si es la primera iteración, no pasamos start_row
    scan_args = {"columns": [b"cf:brand", b"cf:price"], "batch_size": BATCH_SIZE}
    if last_key:
        scan_args["row_start"] = last_key

    # Leer un bloque del escaneo
    rows = list(table.scan(**scan_args, limit=BATCH_SIZE + 1))

    if not rows:
        break

    # Procesar el bloque
    for key, data in rows:
        brand = data.get(b"cf:brand")
        price = data.get(b"cf:price")
        if brand and price:
            try:
                brand_revenue[brand.decode()] += float(price.decode())
            except:
                pass
        last_key = key
        rows_processed += 1

    # Si se leyó menos que el bloque esperado, hemos terminado
    if len(rows) <= BATCH_SIZE:
        break

    print(f"📦 Procesadas {rows_processed:,} filas...")

elapsed_time = time.time() - start_time

if brand_revenue:
    top_brand, top_revenue = brand_revenue.most_common(1)[0]
    print(f"💰 Marca con más ingresos (HBase): {top_brand} con ${top_revenue:,.2f}")
    print(f"⏱ Tiempo total: {elapsed_time:.2f} segundos")
else:
    print("⚠️ No se encontraron datos en la tabla.")


📦 Procesadas 5,001 filas...
📦 Procesadas 10,002 filas...
📦 Procesadas 15,003 filas...
📦 Procesadas 20,004 filas...
📦 Procesadas 25,005 filas...
📦 Procesadas 30,006 filas...
📦 Procesadas 35,007 filas...
📦 Procesadas 40,008 filas...
📦 Procesadas 45,009 filas...
📦 Procesadas 50,010 filas...
📦 Procesadas 55,011 filas...
📦 Procesadas 60,012 filas...
📦 Procesadas 65,013 filas...
📦 Procesadas 70,014 filas...
📦 Procesadas 75,015 filas...
📦 Procesadas 80,016 filas...
📦 Procesadas 85,017 filas...
📦 Procesadas 90,018 filas...
📦 Procesadas 95,019 filas...
📦 Procesadas 100,020 filas...
📦 Procesadas 105,021 filas...
📦 Procesadas 110,022 filas...
📦 Procesadas 115,023 filas...
📦 Procesadas 120,024 filas...
📦 Procesadas 125,025 filas...
📦 Procesadas 130,026 filas...
📦 Procesadas 135,027 filas...
📦 Procesadas 140,028 filas...
📦 Procesadas 145,029 filas...
📦 Procesadas 150,030 filas...
📦 Procesadas 155,031 filas...
📦 Procesadas 160,032 filas...
📦 Procesadas 165,033 filas...
📦 Procesadas 170,034 filas...


📦 Procesadas 1,310,262 filas...
📦 Procesadas 1,315,263 filas...
📦 Procesadas 1,320,264 filas...
📦 Procesadas 1,325,265 filas...
📦 Procesadas 1,330,266 filas...
📦 Procesadas 1,335,267 filas...
📦 Procesadas 1,340,268 filas...
📦 Procesadas 1,345,269 filas...
📦 Procesadas 1,350,270 filas...
📦 Procesadas 1,355,271 filas...
📦 Procesadas 1,360,272 filas...
📦 Procesadas 1,365,273 filas...
📦 Procesadas 1,370,274 filas...
📦 Procesadas 1,375,275 filas...
📦 Procesadas 1,380,276 filas...
📦 Procesadas 1,385,277 filas...
📦 Procesadas 1,390,278 filas...
📦 Procesadas 1,395,279 filas...
📦 Procesadas 1,400,280 filas...
📦 Procesadas 1,405,281 filas...
📦 Procesadas 1,410,282 filas...
📦 Procesadas 1,415,283 filas...
📦 Procesadas 1,420,284 filas...
📦 Procesadas 1,425,285 filas...
📦 Procesadas 1,430,286 filas...
📦 Procesadas 1,435,287 filas...
💰 Marca con más ingresos (HBase): samsung con $54,047,304.62
⏱ Tiempo total: 110.51 segundos


## 3️⃣ Consulta: Mes con más ventas (UTC)

**Objetivo:** Determinar en qué mes (`event_time`) se realizaron más ventas.

Se ejecutará en:
- **MongoDB**
- **Redis**
- **HBase**


In [20]:
start_time = time.time()
pipeline = [
    {"$project": {"mes": {"$substr": ["$event_time", 0, 7]}}},
    {"$group": {"_id": "$mes", "total": {"$sum": 1}}},
    {"$sort": {"total": -1}},
    {"$limit": 1}
]
result = list(mongo_col.aggregate(pipeline))
elapsed_time = time.time() - start_time

print(f"📅 Mes con más ventas (MongoDB): {result[0]['_id']} con {result[0]['total']:,} ventas")
print(f"⏱ Tiempo: {elapsed_time:.4f} segundos")


📅 Mes con más ventas (MongoDB): 2020-06 con 403,632 ventas
⏱ Tiempo: 9.6747 segundos


In [21]:
start_time = time.time()
month_counter = Counter()

for key in r.scan_iter("purchase:*"):
    event_time = r.hget(key, "event_time")
    if event_time:
        month = event_time.decode()[:7]
        month_counter[month] += 1

top_month, top_count = month_counter.most_common(1)[0]
elapsed_time = time.time() - start_time

print(f"📅 Mes con más ventas (Redis): {top_month} con {top_count:,} ventas")
print(f"⏱ Tiempo: {elapsed_time:.4f} segundos")


📅 Mes con más ventas (Redis): 2020-06 con 403,632 ventas
⏱ Tiempo: 1224.5353 segundos


In [37]:
start_time = time.time()
month_counter = Counter()

for key, data in table.scan():
    event_time = data.get(b"cf:event_time")
    if event_time:
        month = event_time.decode()[:7]
        month_counter[month] += 1

top_month, top_count = month_counter.most_common(1)[0]
elapsed_time = time.time() - start_time

print(f"📅 Mes con más ventas (HBase): {top_month} con {top_count:,} ventas")
print(f"⏱ Tiempo: {elapsed_time:.4f} segundos")


📅 Mes con más ventas (HBase): 2020-06 con 211,552 ventas
⏱ Tiempo: 172.2478 segundos
